In [1]:
import import_ipynb #Allows you to import ipynb files
from queue import PriorityQueue
from dataclasses import dataclass

from reedsshepp import *
from vehicle import *
from Grid import Grid
from utils import *

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions(threshold=np.inf)

importing Jupyter notebook from reedsshepp.ipynb
importing Jupyter notebook from vehicle.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from Grid.ipynb
importing Jupyter notebook from GVD.ipynb


In [2]:
@dataclass
class Cell:
    """The cell is the discretized position that the vehicleState falls into"""
    col: int = -1
    row: int = -1
    ori: float = 0.0
    rev: float = 0.0
    def reset(self):
        self.col = -1
        self.row = -1
        self.ori = 0
        self.rev = 0
    def __eq__(self, other):
        return self.col == other.col and self.row == other.row and self.ori == other.ori

In [3]:
#Node class that the A* will be referencing to check and compare costs
class HybridAStarNode:
    
    def __init__(self, parent=None, cell:Cell=None, vehicleState:VehicleState=None, rsAction:ReedsSheppAction=None):
        self.cell = cell
        self.vehicleState = vehicleState #Even though cell and vehicle state store the same thing, cell is discrete and vehiclestate is nondiscrete
        self.reedsSheppAction = rsAction
        self.parent = parent
        self.g = 0 #distance to start node
        self.h = 0 #distance to goal node
        
        self.rsindex = -1
    def FCost(self):
        return self.g + self.h
    
    def __eq__(self, other):
        return self.cell == other.cell
    
    def __lt__(self, other):
        return self.FCost() < other.FCost()
    
    def __gt__(self, other):
        return self.FCost() > other.FCost()
    def __repr__(self):
        return 'Point(x=%s, y=%s, ori=%s, rev=%s)' % (self.cell.col, self.cell.row, np.rad2deg(self.cell.ori), self.cell.rev)

In [4]:
#This will implement the basic A* Algorithm
class HybridAStar:
    #class variables
    def __init__(self, grid:Grid, vehicle:Vehicle):
        self._grid = grid
        self._vehicle = vehicle
    
    def ReedsSheppPath(self, vehicleState, goal):
        pass
    
    #Generates the Path
    def GeneratePath(self, current):
        pass
    
    #Calculates distance heuristic
    def CalculateDistance(self, current, destination):
        return np.sqrt(((current.vehicleState.position[0] - destination.vehicleState.position[0]) ** 2 + (current.vehicleState.position[1] - destination.vehicleState.position[1]) ** 2))
    
    #gets all neighbors at current node
    def GetNextNodes(self, current_state:VehicleState, gear):
        nodes = []
        positions = []
        for steer in self._vehicle.steers:
            newVehicleState, length = self._vehicle.getNextState(current_state, steer, gear, 1)
            #self._grid.IsSafe(newVehicleState, 1.5)
            positions.append((newVehicleState.position, newVehicleState.orientation))
            if(IsPointInGrid(newVehicleState.position, (self._grid.width, self._grid.height))):
                nodes.append(HybridAStarNode(None, None, newVehicleState, ReedsSheppAction(steer, gear, length)))
        #check reeds shepp path
        #add to child if its cheap af
        print(positions)
        return nodes
    
    def statetoCell(self, state:VehicleState):
        x = np.floor(state.position[0])
        y = np.floor(state.position[1])
        theta = mod2Pi(state.orientation)
        rev = 1 if state.gear==Gear.Backward else 0
        return Cell(x, y, theta, rev)
    
    def run(self, start:VehicleState, end:VehicleState):
        start_node = HybridAStarNode(None, self.statetoCell(start), start, ReedsSheppAction(Steer.Straight, Gear.Forward, 0))
        end_node = HybridAStarNode(None, self.statetoCell(end), end)
        
        open_list = PriorityQueue()
        closed_list = []
        open_list.put(start_node)
        voro = self._grid.returnVoroDistanceMap()
        
        while open_list.qsize():
            current_node = open_list.get()
            closed_list.append(current_node)
            if(current_node == end_node):
                return self.GeneratePath(current_node)
            
            for gear in self._vehicle.gears:
                nextNodes = self.GetNextNodes(current_node.vehicleState, gear)
                for nextNode in nextNodes:
                    nextNode.g = current_node.g + 1 #path cost
                    nextNode.h = self.CalculateDistance(nextNode, end_node) #hueristic
                    nextNode.parent = current_node
                    nextNode.cell = self.statetoCell(nextNode.vehicleState)
                    if(nextNode in open_list.queue): continue
                    open_list.put(nextNode)   



In [5]:
g = Grid(31,31)
g.AddObstacle((0,0))
g.AddObstacle((30,30))
g.AddObstacle((15,15))
g.AddObstacle((0,30))
g.AddObstacle((30,0))
g.AddObstacle((15,0))
g.AddObstacle((0,15))
g.AddObstacle((30,15))
g.AddObstacle((15,30))
v = Vehicle((1,1))

In [9]:
hastar = HybridAStar(g, v)
vs1 = VehicleState((.5,10.5))
vs2 = VehicleState((22.5,10.5))
path = hastar.run(vs1, vs2)

[((1.5, 10.5), 0.0), ((1.209996709154892, 11.08834148813814), 1.3839404706095828), ((1.209996709154892, 9.91165851186186), -1.3839404706095828)]
[((-0.5, 10.5), 0.0), ((-0.20999670915489188, 11.08834148813814), -1.3839404706095828), ((-0.20999670915489188, 9.91165851186186), 1.3839404706095828)]
[((2.5, 10.5), 0.0), ((2.209996709154892, 11.08834148813814), 1.3839404706095828), ((2.209996709154892, 9.91165851186186), -1.3839404706095828)]
[((0.5, 10.5), 0.0), ((0.7900032908451081, 11.08834148813814), -1.3839404706095828), ((0.7900032908451081, 9.91165851186186), 1.3839404706095828)]
[((3.5, 10.5), 0.0), ((3.209996709154892, 11.08834148813814), 1.3839404706095828), ((3.209996709154892, 9.91165851186186), -1.3839404706095828)]
[((1.5, 10.5), 0.0), ((1.790003290845108, 11.08834148813814), -1.3839404706095828), ((1.790003290845108, 9.91165851186186), 1.3839404706095828)]
[((4.5, 10.5), 0.0), ((4.2099967091548915, 11.08834148813814), 1.3839404706095828), ((4.2099967091548915, 9.9116585118618